In [3]:
import ROOT
import uproot as ur
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import re
from tqdm import tqdm

In [4]:
os.chdir("/home/das214/FCFD/")
!ls

figures


In [5]:
BASE_PATH = "/depot/cms/users/das214/FCFD/data/"
FIGURES_PATH = "/home/das214/FCFD/figures/"

os.makedirs(FIGURES_PATH, exist_ok=True)

In [6]:
def print_tree(startpath):
    # Ensure the provided path exists
    if not os.path.exists(startpath):
        print(f"Error: Path '{startpath}' does not exist.")
        return

    for root, dirs, files in os.walk(startpath):
        # Calculate the level of indentation
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        
        # Print the current directory
        print(f'{indent}{os.path.basename(root)}/')
        
        # Print the files in the current directory
        sub_indent = ' ' * 4 * (level + 1)
        for f in files:
            print(f'{sub_indent}{f}')

In [7]:
powers = np.arange(80, 95.01, 2.5)  # 95.01 to include 95
powers = powers.tolist()
print(powers)

[80.0, 82.5, 85.0, 87.5, 90.0, 92.5, 95.0]


In [8]:
for power in powers:
    converted_path = os.path.join(BASE_PATH, f"run_Power_{power}%", "converted")
    converted_files = glob.glob(os.path.join(converted_path, "*.root"))
    sorted_files = sorted(
            converted_files,
            key=lambda path: int(re.search(r'run(\d+)\.root', path).group(1))
        )
    print(converted_files)
    print(sorted_files)

['/depot/cms/users/das214/FCFD/data/run_Power_80.0%/converted/converted_run9.root', '/depot/cms/users/das214/FCFD/data/run_Power_80.0%/converted/converted_run6.root', '/depot/cms/users/das214/FCFD/data/run_Power_80.0%/converted/converted_run13.root', '/depot/cms/users/das214/FCFD/data/run_Power_80.0%/converted/converted_run5.root', '/depot/cms/users/das214/FCFD/data/run_Power_80.0%/converted/converted_run4.root', '/depot/cms/users/das214/FCFD/data/run_Power_80.0%/converted/converted_run3.root', '/depot/cms/users/das214/FCFD/data/run_Power_80.0%/converted/converted_run8.root', '/depot/cms/users/das214/FCFD/data/run_Power_80.0%/converted/converted_run14.root', '/depot/cms/users/das214/FCFD/data/run_Power_80.0%/converted/converted_run2.root', '/depot/cms/users/das214/FCFD/data/run_Power_80.0%/converted/converted_run10.root', '/depot/cms/users/das214/FCFD/data/run_Power_80.0%/converted/converted_run12.root', '/depot/cms/users/das214/FCFD/data/run_Power_80.0%/converted/converted_run1.root',

In [9]:
ICHANNEL = 3
X_INIT = 45_910
step_x = 2



In [10]:
all_results_for_combined_plot = []
power = 90.0

converted_path = os.path.join(BASE_PATH, f"run_Power_{power}%", "converted")
converted_files = glob.glob(os.path.join(converted_path, "*.root"))
sorted_files = sorted(
        converted_files,
        key=lambda path: int(re.search(r'run(\d+)\.root', path).group(1))
    )
x_array = []
mean_of_means_array = []
current_x = X_INIT
print(sorted_files)
data_path = sorted_files[0]

with ur.open(data_path) as data_file:
    data = data_file["pulse"]
    out_data = data.arrays(["channel"], library="np")

    SignalMeanOfEvent = []
    num_events = len(out_data["channel"])
    for iEvent in range(num_events):
        # Find where signal is above threshold
        signal_indices = np.where(out_data["channel"][iEvent, ICHANNEL, :] > 0.1)[0]
        first_index = signal_indices[0]
        last_index = signal_indices[-1]

        # Calculate mean, avoiding edges
        if last_index - first_index > 20:
            mean_in_region = np.mean(out_data["channel"][iEvent, ICHANNEL, first_index + 10 : last_index - 10])
            SignalMeanOfEvent.append(mean_in_region)

    # Calculate the final mean for this file
    if SignalMeanOfEvent:
        mean_value = sum(SignalMeanOfEvent) / len(SignalMeanOfEvent)
        mean_of_means_array.append(mean_value)
        x_array.append(current_x)




['/depot/cms/users/das214/FCFD/data/run_Power_90.0%/converted/converted_run1.root', '/depot/cms/users/das214/FCFD/data/run_Power_90.0%/converted/converted_run2.root', '/depot/cms/users/das214/FCFD/data/run_Power_90.0%/converted/converted_run3.root', '/depot/cms/users/das214/FCFD/data/run_Power_90.0%/converted/converted_run4.root', '/depot/cms/users/das214/FCFD/data/run_Power_90.0%/converted/converted_run5.root', '/depot/cms/users/das214/FCFD/data/run_Power_90.0%/converted/converted_run6.root', '/depot/cms/users/das214/FCFD/data/run_Power_90.0%/converted/converted_run7.root', '/depot/cms/users/das214/FCFD/data/run_Power_90.0%/converted/converted_run8.root', '/depot/cms/users/das214/FCFD/data/run_Power_90.0%/converted/converted_run9.root', '/depot/cms/users/das214/FCFD/data/run_Power_90.0%/converted/converted_run10.root', '/depot/cms/users/das214/FCFD/data/run_Power_90.0%/converted/converted_run11.root', '/depot/cms/users/das214/FCFD/data/run_Power_90.0%/converted/converted_run12.root', 

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
for ICHANNEL in range(7):
    y_waveform = out_data["channel"][iEvent, ICHANNEL, :]
    x_samples = np.arange(len(y_waveform)) # Create an x-axis for plotting

    plt.plot(x_samples, y_waveform, label = f"Channel: {ICHANNEL+1}")
    plt.ylim(-0.085, 0.5)
    
plt.legend()
plt.xlabel("Samples")
plt.ylabel("Voltage [V]")
plt.ylim(-0.085, 0.1)
plt.show()

In [21]:
ICHANNEL = 0
iEvent = 0
y_waveform = out_data["channel"][iEvent, ICHANNEL, :]
x_samples = np.arange(len(y_waveform)) # Create an x-axis for plotting

plt.plot(x_samples, y_waveform)
plt.ylim(-0.1, 0.5)
    
plt.legend()
plt.xlabel("Samples")
plt.ylabel("Voltage [V]")
plt.show()
    

/tmp/ipykernel_445365/2726602320.py:9: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


In [22]:
import numpy as np
import matplotlib.pyplot as plt

# Given:
# y_waveform = out_data["channel"][iEvent, ICHANNEL, :]
# If you know scope sampling: dt = 0.5e-9  # e.g., 0.5 ns per sample
dt = 1.0  # change to the real sampling interval (seconds)

# Time axis (optional, for plotting in seconds)
t = np.arange(y_waveform.shape[0]) * dt

# --- High-quality 2nd-order derivative (C-optimized) ---
dy_dt = np.gradient(y_waveform.astype(np.float64, copy=False), dt, edge_order=2)

# (Alternative: explicit central differences, same result, slightly more control)
# dy_dt = np.empty_like(y_waveform, dtype=np.float64)
# dy_dt[1:-1] = (y_waveform[2:] - y_waveform[:-2]) * (0.5/dt)
# dy_dt[0]    = (y_waveform[1] - y_waveform[0]) / dt
# dy_dt[-1]   = (y_waveform[-1] - y_waveform[-2]) / dt

# Plot
fig, ax = plt.subplots()
ax.plot(t, y_waveform, label="Voltage")
ax.set_ylim(-0.1, 0.5)
ax.set_xlabel("Time [s]" if dt != 1.0 else "Samples")
ax.set_ylabel("Voltage [V]")

ax2 = ax.twinx()
ax2.plot(t, dy_dt, ls="--", alpha=0.9, label="dV/dt")
ax2.set_ylabel("dV/dt [V/s]" if dt != 1.0 else "Î”V per sample")

# Combined legend
lines = ax.get_lines() + ax2.get_lines()
labels = [ln.get_label() for ln in lines]
ax.legend(lines, labels, loc="best")

plt.show()


In [1]:
ICHANNEL = 1

for ICHANNEL in range(7):
    y_waveform = out_data["channel"][iEvent, ICHANNEL, :]
    x_samples = np.arange(len(y_waveform)) # Create an x-axis for plotting

    plt.plot(x_samples, y_waveform, label = f"Channel: {ICHANNEL+1}")
    plt.ylim(-0.01, 0.085)
    
plt.legend()
plt.xlabel("Samples")
plt.ylabel("Voltage [V]")
plt.show()

NameError: name 'out_data' is not defined

In [ ]:
# Intelligently find the first event with a pulse and plot it

with ur.open(data_path) as data_file:
    data = data_file["pulse"]
    out_data = data.arrays(["channel"], library="np")
    
    found_event_to_plot = -1
    num_events = len(out_data["channel"])

    for iEvent in range(num_events):
        # Your pulse-finding logic
        signal_indices = np.where(out_data["channel"][iEvent, ICHANNEL, :] > 0.1)[0]
        if len(signal_indices) > 0:
            # We found an event that crosses the threshold!
            print(f"Found a potential pulse in Event {iEvent}. Plotting it.")
            found_event_to_plot = iEvent
            break # Exit the loop since we found one
            
    if found_event_to_plot != -1:
        y_waveform = out_data["channel"][found_event_to_plot, ICHANNEL, :]
        x_samples = np.arange(len(y_waveform))

        plt.figure(figsize=(10, 6))
        plt.plot(x_samples, y_waveform)
        plt.title(f'Waveform for First Found Pulse (Event {found_event_to_plot}), Channel {ICHANNEL+1}')
        plt.xlabel("Sample Number")
        plt.ylabel("Voltage [V]")
        plt.grid(True)
        plt.show()
    else:
        print(f"Could not find any event with a pulse above 0.1 V in {data_path}")

Could not find any event with a pulse above 0.1 V in /depot/cms/users/das214/FCFD/data/run_Power_80.0%/converted/converted_run1.root


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter